In [ ]:
import gc
from tqdm import tqdm
import numpy as np
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.stats import probplot

import datetime as dt
from datetime import date
from datetime import timedelta
from fbprophet import Prophet

%matplotlib inline
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data0=pd.read_csv('../input/earthquake-in-japan/japan_20190101-20211009_query.csv')
data0[1:2]

In [ ]:
data0=data0.fillna('N')
data1=data0[['time','latitude','longitude','depth','mag',]]
data1[0:5]

In [ ]:
data2=pd.DataFrame(columns=data1.columns.tolist())
for i,item in enumerate(data1['time']):
    data2=pd.concat([data2,data1.loc[i:i]])
data2=data2.sort_values(['time']).reset_index(drop=True)    

In [ ]:
from datetime import datetime as dt
data2a=data2.copy()
date=[]
for item in data2['time']:
    date+=[dt.strptime(item[0:-5],'%Y-%m-%dT%H:%M:%S')]
data2a['date']=date
data2a

In [ ]:
data4=data2a[data2a['latitude']<36.3][data2a['latitude']>34.8][data2a['longitude']>138.7][data2a['longitude']<140.9]
data4=data4.reset_index(drop=True)
data4

In [ ]:
import plotly.express as px
fig=px.scatter(x=data4['date'],y=data4['mag'])
fig=px.scatter(data4, x="date", y="mag")
fig.update_layout(title='Magnituide of earthquake around Tokyo in 2019-2021')
fig.show()

In [ ]:
data4['date2']=data4['date'].apply(lambda x: dt.strptime(str(x)[0:10],'%Y-%m-%d') )
data4

In [ ]:
data5=data4[['date2','mag']]
data5

In [ ]:
item1=data5

item1.columns = ['ds','y']
item1.y = item1.y.astype('float')
item1.ds = item1.ds.astype('datetime64')

rcParams['figure.figsize'] = 20, 5
plt.plot(item1.ds, item1.y)

# fit past and predict past

In [ ]:
ph = Prophet()
ph.fit(item1)
forecast1=ph.predict(item1)
figure = ph.plot(forecast1)
figure.show()

# fit past and predict past+future

In [ ]:
from datetime import timedelta
start0 = dt.strptime('2021-01-01', '%Y-%m-%d').date()
end0   = dt.strptime('2030-12-31', '%Y-%m-%d').date()
print((end0-start0).days)

In [ ]:
def daterange(start,end):
    for i in range((end-start).days):
        return start+timedelta(i)  

In [ ]:
dates0=[]
for i in range((end0-start0).days):
    dates0+=[(start0+timedelta(i)).strftime('%Y-%m-%d') ]
print(dates0[0:30])

In [ ]:
dates0_df=pd.DataFrame(dates0)
dates0_df.columns=['ds']
dates0_df

In [ ]:
ph = Prophet()
ph.fit(item1)
forecast3=ph.predict(dates0_df)
figure = ph.plot(forecast3)
figure.show()

### In Tokyo, the magnitude of the earthquake is likely to increase gradually in the future. In addition, earthquakes above the predicted range, like M5.9 erthquake on 2021-10-7, might occur again.

cf. https://www.kaggle.com/stpeteishii/m5-9-earthquake-in-tokyo-on-2021-10-07